First crack at taking lat/long and finding the zip code 8/3/2025

Here's my first prompt: in python, how would I convert latitude and longitude position into a zip code?
To convert a latitude and longitude into a ZIP code in Python, you'll typically use reverse geocoding. The most common way to do this is with a geocoding API, such as:
Google Maps Geocoding API (paid, accurate)
OpenStreetMap / Nominatim (free, decent)
Example using geopy with Nominatim (Free and easy)

In [2]:
from geopy.geocoders import Nominatim

# Initialize the geocoder
geolocator = Nominatim(user_agent="zip_code_locator")

# Example coordinates
latitude = 45.493713
longitude = -122.576362


# Perform reverse geocoding
location = geolocator.reverse((latitude, longitude), exactly_one=True)

# Extract ZIP code
if location and 'postcode' in location.raw['address']:
    zip_code = location.raw['address']['postcode']
    print(f"ZIP code: {zip_code}")
else:
    print("ZIP code not found.")

ZIP code: 97266


Now let's see if we can get a portland neighborhood from lat/long. Prompt:
I would like a python script that takes latitude and longitude and returns the name of the portland, oregon neighborhood that location is in.


In [10]:
from geopy.geocoders import Nominatim

def get_neighborhood(lat, lon):
    geolocator = Nominatim(user_agent="pdx_neighborhood_finder")
    location = geolocator.reverse((lat, lon), exactly_one=True)
    
    if location:
        address = location.raw.get('address', {})
        # Neighborhood is often in 'neighbourhood' or 'suburb'
        return address.get('neighbourhood') or address.get('suburb') or "Neighborhood not found"
    else:
        return "Location not found"

# Example: Alberta Arts District
# 5028 NE 13th ave 45.559913, -122.652339
lat = 45.559913
lon = -122.652339
# from our IRP post sites file:
# -13642031.74	5696806.287
# ok, so those numbers aren't lat/long.  now, what are they?  see below for a script to convert them

neighborhood = get_neighborhood(lat, lon)
print(f"Neighborhood: {neighborhood}")


Neighborhood: King


Ok, now to find neighborhood by street address.  Prompt: 
I need a python script that takes a street address like "5028 ne 13th ave" and returns the name of the portland, oregon neighborhood that it is in.
Got it. To accurately get the Portland neighborhood for an address like "5028 NE 13th Ave", here's a two-part Python solution:
Step 1: Geocode the address to get latitude and longitude
We'll use Nominatim (OpenStreetMap) for free geocoding.
Step 2: Load Portland’s official neighborhood boundaries
We'll download Portland’s official neighborhood shapefile and use geopandas to match the lat/lon to the correct neighborhood polygon.
(if inside a jupyter notebook !pip install geopy)
--> pip install geopy geopandas shapely
--> pip install fiona pyproj
Download Portland Neighborhood Shapefile (we have that)


In [5]:
!pip install geopy geopandas shapely
!pip install fiona pyproj
print("modules installed!")

   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 1.7/1.7 MB 15.4 MB/s  0:00:00
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ----------------------------- ---------- 8.1/11.0 MB 40.7 MB/s eta 0:00:01
   ---------------------------------------- 11.0/11.0 MB 34.3 MB/s  0:00:00
   ---------------------------------------- 0.0/19.2 MB ? eta -:--:--
   ---------------- ----------------------- 7.9/19.2 MB 42.1 MB/s eta 0:00:01
   --------------------------------- ------ 16.0/19.2 MB 40.4 MB/s eta 0:00:01
   ---------------------------------------- 19.2/19.2 MB 35.7 MB/s  0:00:00
   ---------------------------------------- 0.0/6.3 MB ? eta -:--:--
   ---------------------------------------- 6.3/6.3 MB 32.8 MB/s  0:00:00

   ---------------------------------------- 0/7 [pytz]
   ---------------------------------------- 0/7 [pytz]
   ---------------------------------------- 0/7 [pytz]
   ------------------

In [7]:
from geopy.geocoders import Nominatim
import geopandas as gpd
from shapely.geometry import Point

# Load Portland neighborhood shapefile
# Replace this path with where you extracted the shapefile
NEIGHBORHOOD_SHP_PATH = "datasets/original/Neighborhoods_regions/Neighborhoods_regions.shp"

def get_lat_lon(address, city="Portland", state="OR"):
    geolocator = Nominatim(user_agent="pdx_neighborhood_locator")
    location = geolocator.geocode(f"{address}, {city}, {state}")
    if location:
        return location.latitude, location.longitude
    else:
        return None, None

def get_neighborhood(address):
    lat, lon = get_lat_lon(address)
    if not lat:
        return "Address could not be geocoded."

    point = Point(lon, lat)

    # Load the shapefile with geopandas
    neighborhoods = gpd.read_file(NEIGHBORHOOD_SHP_PATH)

    # Make sure CRS matches (use EPSG:4326 for lat/lon)
    if neighborhoods.crs != "EPSG:4326":
        neighborhoods = neighborhoods.to_crs(epsg=4326)

    # Find the neighborhood containing the point
    match = neighborhoods[neighborhoods.contains(point)]
    
    if not match.empty:
        return match.iloc[0]["NAME"]  # Column may be 'NAME' or similar
    else:
        return "Neighborhood not found in boundary data."

# Example usage
address = "5028 NE 13th Ave"
neighborhood = get_neighborhood(address)
print(f"{address} is in: {neighborhood}")


5028 NE 13th Ave is in: KING


So, to get a count of homeless we're going to need to use our IRP Posts data.
This has x,y coords that aren't lat/long.  So, here's how to convert them:
"The coordinates you have—like -13642031.74, 5696806.287—are projected coordinates, not latitude/longitude. They are most likely in the Web Mercator projection (EPSG:3857), commonly used for maps and GIS data, especially for areas like Portland."
Convert Web Mercator (EPSG:3857) to Latitude/Longitude (EPSG:4326) in Python
You can use pyproj to convert them:
pip install pyproj

In [11]:
!pip install pyproj
print("done installing!")

In [12]:
from pyproj import Transformer

# Create a transformer from EPSG:3857 to EPSG:4326
transformer = Transformer.from_crs("EPSG:3857", "EPSG:4326", always_xy=True)

# Your X and Y coordinates (Web Mercator)
x = -13642031.74
y = 5696806.287

# Convert to lon/lat
lon, lat = transformer.transform(x, y)
print(f"Latitude: {lat}, Longitude: {lon}")


Latitude: 45.47621706253292, Longitude: -122.54845618485629


In [1]:
!pip install pandas geopy
print("done installing!")

done installing!


In [ ]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

# Input and output CSV paths
input_csv = "datasets/original/project_data/IRP_Post_Sites_with_LatLon.csv"
output_csv = "datasets/processed/IRP_Post_Sites_with_lat_long_Zip_Neighborhood.csv"

# Load input CSV
df = pd.read_csv(input_csv)

# Initialize geolocator with rate limiter
geolocator = Nominatim(user_agent="pdx_geo_enricher")
geocode = RateLimiter(geolocator.reverse, min_delay_seconds=1)

# Function to extract ZIP and neighborhood
def get_location_info(lat, lon):
    try:
        location = geocode((lat, lon), exactly_one=True, addressdetails=True)
        if location and "address" in location.raw:
            address = location.raw["address"]
            zip_code = address.get("postcode", "")
            neighborhood = address.get("neighbourhood") or address.get("suburb") or address.get("city_district", "")
            return pd.Series([zip_code, neighborhood])
    except Exception:
        return pd.Series(["", ""])
    return pd.Series(["", ""])

# Apply function to DataFrame
df[['ZIP_Code', 'Neighborhood']] = df.apply(
    lambda row: get_location_info(row['Latitude'], row['Longitude']), axis=1
)

# Save updated DataFrame
df.to_csv(output_csv, index=False)
print(f"Saved enriched file to {output_csv}")
